<a href="https://colab.research.google.com/github/Adammac7/BlackJack/blob/main/Draft_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Colab setup: pin versions to avoid ABI conflicts
!pip install -qU "numpy==2.0.2" "pandas==2.2.2" \
                 "pyarrow>=15" "fastparquet>=2024.5.0" \
                 "nfl_data_py>=0.3.0"

print("✅ Installed. Now go to: Runtime → Restart runtime, then run the next cell.")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 65.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 25.6.0 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 21.0.0 which is incompatible.
pylibcudf-cu12 25.6.0 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 21.0.0 which is incompatible.
✅ Installed. Now go to: Runtime → Restart runtime, then run the next cell.


In [3]:
# Install nfl_data_py in Colab
!pip install nfl_data_py

# Import the library
import nfl_data_py
import pandas as pd

In [4]:
import pandas as pd
from dataclasses import dataclass, field
import random
import math

# =========================
# CONFIG
# =========================
NUM_TEAMS = 12
ROSTER_LIMITS = {"QB":1, "RB":2, "WR":2, "TE":1, "FLEX":1, "BENCH":5}   # total 12
FLEX_ELIG = {"RB","WR","TE"}
ROUNDS = sum(ROSTER_LIMITS.values())
# New config for draft randomness
RANDOM_PICK_RANGE = 8 # Number of top available players to consider for a random pick with weighted probability

# =========================
# INPUT: players_by_adp  (BEST → WORST by ADP)
# This should now be the 'aligned_players' list of dictionaries
# =========================

def to_dataframe(player_list):
    """Converts a list of player dictionaries or tuples to a pandas DataFrame."""
    if not player_list:
        raise ValueError("players_by_adp is empty.")
    # Assuming player_list is a list of dictionaries with consistent keys
    df = pd.DataFrame(player_list)
    # Ensure required columns exist
    required_cols = ["original_name", "pos"] # Use original_name as the primary identifier now
    if not all(col in df.columns for col in required_cols):
         raise ValueError(f"Each player dict must contain at least {required_cols}.")

    # Select and reorder columns to match expected structure for sorting
    cols = [c for c in ["original_name", "full_name", "player_id", "pos", "team", "proj", "adp", "bye"] if c in df.columns]
    return df[cols].copy()

@dataclass
class Team:
    roster: list = field(default_factory=list)
    # backups_needed and starting_filled are no longer used directly in run_snake_draft
    # but kept for potential future use or clarity if needed elsewhere.
    backups_needed: dict = field(default_factory=lambda: {"QB":1,"RB":1,"WR":1,"TE":1})
    starting_filled: bool = False


def count_by_pos(roster):
    cnt = {"QB":0,"RB":0,"WR":0,"TE":0,"FLEX":0,"BENCH":0}
    starters, flex_pool = [], []
    for p in roster:
        # Use get() for safety in case 'pos' is missing, though it should be present in aligned_players
        pos = p.get("pos")
        if pos in ["QB","RB","WR","TE"]:
            starters.append(p)
    for p in starters:
        pos = p.get("pos")
        if pos in ["RB","WR","TE"]:
            if cnt[pos] < ROSTER_LIMITS.get(pos, 0):
                cnt[pos]+=1
            else:
                flex_pool.append(p)
        elif pos=="QB":
            if cnt["QB"] < ROSTER_LIMITS.get("QB", 0):
                cnt["QB"]+=1
            else:
                flex_pool.append(p)
    for p in flex_pool:
        if p.get("pos") in FLEX_ELIG and cnt["FLEX"] < ROSTER_LIMITS.get("FLEX", 0):
            cnt["FLEX"]+=1
        else:
            cnt["BENCH"]+=1
    starter_cap = sum(ROSTER_LIMITS.values()) - ROSTER_LIMITS.get("BENCH", 0)
    cnt["BENCH"] = max(0, len(roster) - starter_cap)
    return cnt


def starting_needs(team, roster_limits, flex_elig):
    cnt = count_by_pos(team.roster)
    needs = set()
    if cnt.get("QB", 0) < roster_limits.get("QB", 0): needs.add("QB")
    if cnt.get("RB", 0) < roster_limits.get("RB", 0): needs.add("RB")
    if cnt.get("WR", 0) < roster_limits.get("WR", 0): needs.add("WR")
    if cnt.get("TE", 0) < roster_limits.get("TE", 0): needs.add("TE")
    fixed_full = (cnt.get("RB", 0) >= roster_limits.get("RB", 0) and cnt.get("WR", 0) >= roster_limits.get("WR", 0) and cnt.get("TE", 0) >= roster_limits.get("TE", 0))
    if cnt.get("FLEX", 0) < roster_limits.get("FLEX", 0) and fixed_full:
        needs.update(flex_elig)
    return needs

def is_starting_filled(team, roster_limits):
    cnt = count_by_pos(team.roster)
    return (
        cnt.get("QB", 0)  >= roster_limits.get("QB", 0)  and
        cnt.get("RB", 0)  >= roster_limits.get("RB", 0)  and
        cnt.get("WR", 0)  >= roster_limits.get("WR", 0)  and
        cnt.get("TE", 0)  >= roster_limits.get("TE", 0)  and
        cnt.get("FLEX", 0) >= roster_limits.get("FLEX", 0)
    )


def pick_with_weighted_probability(df_avail, pick_range=RANDOM_PICK_RANGE):
    """
    Picks a player from the top available players within a range using weighted probability.
    Weights are based on inverse of their rank within the pick_range.
    """
    top_available = df_avail.head(pick_range)
    if top_available.empty:
        return None

    # Calculate weights based on rank (position in the top_available subset)
    # Use the values of the reset index, which represent the rank (starting from 0)
    weights = 1 / (top_available.reset_index().index.values + 1)

    # Normalize weights to sum to 1
    normalized_weights = weights / weights.sum()

    # Randomly pick an index from the original df_avail based on weights
    # Use top_available.index to get the actual indices from the original DataFrame
    picked_index_in_df_avail = random.choices(top_available.index, weights=normalized_weights, k=1)[0]

    return picked_index_in_df_avail


def pick_best_for_positions_weighted(df_avail, allowed_positions, pick_range=RANDOM_PICK_RANGE):
    """
    Picks the best available player from allowed positions, with weighted randomness
    among the top eligible players within the pick_range.
    """
    # Filter available players by allowed positions
    eligible_players = df_avail[df_avail['pos'].isin(allowed_positions)]
    if eligible_players.empty:
        return None

    # Consider the top eligible players within the defined pick_range
    top_eligible = eligible_players.head(pick_range)
    if top_eligible.empty:
         # Fallback to picking the very next eligible player if pick_range is too small
         # This can happen if there aren't enough eligible players in the available pool
         if not eligible_players.empty:
              return eligible_players.index[0]
         else:
              return None # No eligible players at all


    # Calculate weights based on rank within the top_eligible subset
    # Use the values of the reset index, which represent the rank (starting from 0)
    weights = 1 / (top_eligible.reset_index().index.values + 1)
    normalized_weights = weights / weights.sum()

    # Randomly pick an index from the original df_avail based on weights
    # Use top_eligible.index to get the actual indices from the original DataFrame
    picked_index_in_eligible = random.choices(top_eligible.index, weights=normalized_weights, k=1)[0]

    return picked_index_in_eligible


def snake_order(round_num, num_teams):
    return list(range(num_teams)) if (round_num % 2 == 0) else list(range(num_teams))[::-1]

def run_snake_draft(players_by_adp, num_teams=NUM_TEAMS, roster_limits=ROSTER_LIMITS, flex_elig=FLEX_ELIG, rounds=ROUNDS, random_pick_range=RANDOM_PICK_RANGE):
    """
    Simulates a snake draft based on player ADP.
    Returns a list of Team objects with drafted players (as dictionaries)
    and a log of picks.
    """
    picks_log = [] # Initialize picks_log here
    # players_by_adp is expected to be a list of dictionaries (aligned_players)
    df = to_dataframe(players_by_adp).copy()
    if "adp" in df.columns:
        # Sort by ADP, placing NaN at the end
        df = df.sort_values("adp", kind="mergesort", na_position="last").reset_index(drop=True)
    else:
        # If no ADP, sort by projection as a fallback
        df = df.sort_values("proj", kind="mergesort", ascending=False).reset_index(drop=True)

    df["draft_order"] = range(len(df)) # Assign a draft order based on the initial sorted list


    teams = [Team() for _ in range(num_teams)]
    taken_indices = set() # Store indices from the df_draftable DataFrame


    overall = 0
    for rnd in range(rounds):
        order = snake_order(rnd, num_teams)
        for team_idx in order:
            # Filter out players already taken using the original DataFrame indices
            df_avail = df[~df.index.isin(taken_indices)].sort_values("draft_order").copy()
            if df_avail.empty: break

            team = teams[team_idx]
            # Use weighted random picking strategy
            if not is_starting_filled(team, roster_limits):
                needs = starting_needs(team, roster_limits, flex_elig)
                # If needs are not empty, try to pick the best for needed positions with weighted randomness
                if needs:
                    pick_idx = pick_best_for_positions_weighted(df_avail, list(needs), random_pick_range)
                else:
                     # If starting lineup is not filled but no specific position is needed (e.g., only FLEX needed, but no eligible players left)
                     # Fallback to general weighted random pick from all available
                     pick_idx = pick_with_weighted_probability(df_avail, random_pick_range)

            else:
                # Apply weighted randomness to all available players for bench spots
                pick_idx = pick_with_weighted_probability(df_avail, random_pick_range)


            # Ensure a player index was selected
            if pick_idx is None:
                 # Fallback to picking the very next available player if random selection failed or no specific need met
                 df_avail_fallback = df[~df.index.isin(taken_indices)].sort_values("draft_order")
                 if not df_avail_fallback.empty:
                      pick_idx = df_avail_fallback.index[0]
                 else:
                      break # No players left


            # Get the drafted player's information as a dictionary
            # Ensure drafted_player_info includes 'original_name' and 'player_id'
            drafted_player_info = df.loc[pick_idx].to_dict()

            # Add the player's dictionary to the team's roster
            teams[team_idx].roster.append(drafted_player_info)

            # Mark the player as taken using their index in the original DataFrame
            taken_indices.add(pick_idx)

            # Recalculate starting_filled and backups_needed after each pick (optional, mainly for strategy logic)
            teams[team_idx].starting_filled = is_starting_filled(teams[team_idx], roster_limits)

            # Log the pick
            picks_log.append({
                "round": rnd+1,
                "overall": overall+1,
                "team": team_idx+1,
                "name": drafted_player_info.get("original_name", drafted_player_info.get("name", "Unknown")), # Use original_name if available
                "pos": drafted_player_info.get("pos", "N/A"),
                "nfl": drafted_player_info.get("team",""),
            })
            overall += 1

    return teams, picks_log


def print_all_teams(teams):
    for i, team in enumerate(teams, start=1):
        print(f"\n=== Team {i} ===")
        df_team = pd.DataFrame(team.roster)
        # Use 'original_name' for display if available, fallback to 'name'
        cols = [c for c in ["original_name","pos","team","adp","proj","bye"] if c in df_team.columns]
        if "original_name" in df_team.columns:
             df_team = df_team.rename(columns={"original_name": "name"})
             cols[cols.index("original_name")] = "name" # Update cols list as well


        if df_team.empty:
            print("(no picks)"); continue
        print(df_team[cols].to_string(index=False))


def print_draft_board(picks_log, num_teams=NUM_TEAMS, rounds=ROUNDS, show_pos=True):
    """
    Prints a grid: rows = rounds, cols = Team 1..Team N.
    Each cell = 'Player (POS)' drafted by that team in that round.
    """
    # build empty board
    board = [[ "" for _ in range(num_teams)] for _ in range(rounds)]
    for p in picks_log:
        r = p["round"] - 1
        t = p["team"]  - 1
        label = p["name"] if not show_pos else f"{p['name']} ({p['pos']})"
        # Ensure indices are within bounds
        if 0 <= r < rounds and 0 <= t < num_teams:
            board[r][t] = label
        else:
            print(f"Warning: Pick log out of bounds - Round {p['round']}, Team {p['team']}")


    # pretty print using pandas
    df_board = pd.DataFrame(board, index=[f"Rnd {i}" for i in range(1, rounds+1)],
                            columns=[f"Team {j}" for j in range(1, num_teams+1)])
    print("\n=== Draft Board ===")
    # Use to_string to ensure full board is printed without truncation
    print(df_board.to_string())


# =========================
# USAGE
# =========================
# To run the draft:
# teams, picks = run_snake_draft(aligned_players) # Pass the aligned_players list
# print_draft_board(picks)      # <-- draft board grid
# print_all_teams(teams)        # <-- team-by-team rosters

In [5]:
import pandas as pd
import numpy as np
import math
import random
from dataclasses import dataclass, field
from collections import defaultdict

# Assuming nfl_data_py is already installed and imported in the environment

@dataclass
class Team:
    roster: list = field(default_factory=list)

def run_full_simulation(
    num_teams: int = 12,
    roster_limits: dict = None,
    total_weeks: int = 15,
    my_slot: int = 1, # This parameter is not currently used in the simulation logic
    projections_path: str = "/content/drive/MyDrive/football/FF_2023_Projections.csv",
    adp_path: str = "/content/drive/MyDrive/football/FantasyPros_2023_Overall_ADP_Rankings.csv",
    weekly_stats_year: int = 2023,
    injuries_year: int = 2023

):
    """
    Runs a full fantasy football season simulation including draft, schedule,
    weekly matchups with coach bot, and final standings.

    Args:
        num_teams (int): The number of teams in the league.
        roster_limits (dict): Dictionary defining roster limits by position.
        total_weeks (int): The total number of weeks to simulate.
        my_slot (int): The user's draft slot (1-based).
        projections_path (str): Path to the projections CSV file.
        adp_path (str): Path to the ADP CSV file.
        weekly_stats_year (int): The year for weekly statistics data.
        injuries_year (int): The year for injury data.

    Returns:
        tuple: A tuple containing:
               - dict: The final standings.
               - list: The list of Team objects with drafted rosters (including 'started_weeks').
               - list: The generated schedule.
               - defaultdict: The weekly PPR points map.
               - dict: The weekly player availability map.
               - list: A list of dictionaries, each representing a weekly matchup with starting players and scores.
    """
    if roster_limits is None:
        roster_limits = {"QB":1, "RB":2, "WR":2, "TE":1, "FLEX":1, "BENCH":5}
    FLEX_ELIG = {"RB","WR","TE"}
    ROUNDS = sum(roster_limits.values())
    RANDOM_PICK_RANGE = 8 # Defined within the simulation function now


    print("--- Starting Fantasy Football Simulation ---")

    # --- Data Loading and Preparation ---
    print("Loading and preparing data...")
    try:
        projections_df = pd.read_csv(projections_path)
        adp_df = pd.read_csv(adp_path)
        merged_df = pd.merge(projections_df, adp_df, on='Player', how='left')
        player_data = merged_df[['Player', 'POS_x', 'Team_x', 'FPTS', 'AVG', 'Bye']]
        player_data = player_data.rename(columns={'Player': 'name', 'POS_x': 'pos', 'Team_x': 'team', 'FPTS': 'proj', 'AVG': 'adp', 'Bye': 'bye'})
        player_list = list(player_data.itertuples(index=False, name=None))
        weeks_started = dict()

        def adp_key(t):
            adp = t[4]
            try:
                adp = float(adp)
            except (TypeError, ValueError):
                return float('inf')
            return adp if not math.isnan(adp) else float('inf')
        sorted_player_list = sorted(player_list, key=adp_key)

        # Import weekly data and player info for name alignment and stats
        # Using weekly_stats_year for data import
        weekly_stats_df = nfl_data_py.import_weekly_data(years=[weekly_stats_year])
        players_df = nfl_data_py.import_players()
        players_df = players_df.rename(columns={'gsis_id': 'player_id'})

        # Create robust name to ID mapping and display_name to full_name mapping
        name_to_id_robust = {}
        display_name_to_full_name = {}
        if not players_df.empty:
            for index, row in players_df.iterrows():
                player_id = row.get('player_id')
                display_name = row.get('display_name')
                full_name = row.get('player_name')

                if pd.notna(player_id):
                    if pd.notna(display_name):
                        name_to_id_robust[display_name] = player_id
                    if pd.notna(full_name):
                        if full_name not in name_to_id_robust: # Prioritize display name
                            name_to_id_robust[full_name] = player_id
                    # Add common_first_name.last_name as a possible key
                    if pd.notna(row.get('common_first_name')) and pd.notna(row.get('last_name')):
                         simple_name = f"{row['common_first_name'][0]}.{row['last_name']}"
                         if simple_name not in name_to_id_robust: # Prioritize display name and full name
                             name_to_id_robust[simple_name] = player_id


                if pd.notna(display_name) and pd.notna(full_name):
                     display_name_to_full_name[display_name] = full_name


        # Create aligned_players list with player_id and full_name
        # Use the player_data DataFrame (derived from projections and adp) as the base
        # and add 'player_id' and 'full_name'
        aligned_players = []
        # Convert player_data DataFrame to a list of dictionaries to work with
        player_data_list = player_data.to_dict('records')


        for player_info in player_data_list:
             player_name_in_list = player_info.get('name')
             if player_name_in_list:
                 player_id = name_to_id_robust.get(player_name_in_list)
                 full_name = display_name_to_full_name.get(player_name_in_list, player_name_in_list) # Use mapping, fallback to original name

                 aligned_player_dict = {
                     'original_name': player_name_in_list,
                     'full_name': full_name,
                     'player_id': player_id,
                     'pos': player_info.get('pos'),
                     'team': player_info.get('team'),
                     'proj': player_info.get('proj'),
                     'adp': player_info.get('adp'),
                     'bye': player_info.get('bye')
                 }
                 aligned_players.append(aligned_player_dict)


        # Sort aligned_players by adp for the draft simulation
        # Create a temporary DataFrame from aligned_players for sorting
        df_aligned_for_sort = pd.DataFrame(aligned_players)
        if 'adp' in df_aligned_for_sort.columns:
            df_aligned_for_sort = df_aligned_for_sort.sort_values("adp", kind="mergesort", na_position="last").reset_index(drop=True)
        elif 'proj' in df_aligned_for_sort.columns:
             # Fallback to sorting by projection if ADP is not available
             df_aligned_for_sort = df_aligned_for_sort.sort_values("proj", kind="mergesort", ascending=False).reset_index(drop=True)
        else:
             # If neither ADP nor proj is available, keep original order
             pass


        aligned_players_sorted_by_adp = df_aligned_for_sort.to_dict('records')


        # Filter weekly stats for drafted players (using player_id from aligned_players)
        # Ensure drafted_player_ids is created from the aligned_players_sorted_by_adp list
        drafted_player_ids = {p.get('player_id') for p in aligned_players_sorted_by_adp if p.get('player_id')}


        weekly_stats_drafted = weekly_stats_df[weekly_stats_df['player_id'].isin(drafted_player_ids)].copy() # Use .copy() to avoid SettingWithCopyWarning

        # Cap weekly stats to total_weeks
        weekly_stats_drafted = weekly_stats_drafted[weekly_stats_drafted['week'] <= total_weeks]


        weekly_ppr_points_map = defaultdict(lambda: defaultdict(float)) # (player_id, week) -> points
        if not weekly_stats_drafted.empty:
            for index, row in weekly_stats_drafted.iterrows():
                 if pd.notna(row.get('player_id')) and pd.notna(row.get('week')):
                    weekly_ppr_points_map[row['player_id']][row['week']] = row.get('fantasy_points_ppr', 0.0)


        # Import injury data using injuries_year
        injuries_df = nfl_data_py.import_injuries([injuries_year])
        # Use original_name from aligned_players for bye weeks
        player_bye_weeks = {player.get('original_name'): player.get('bye') for player in aligned_players if pd.notna(player.get('bye'))}


        player_injury_status = {}
        if not injuries_df.empty:
            for index, row in injuries_df.iterrows():
                # Try to get the player name from the injury data first (full_name or display_name)
                player_name_injury = row.get('full_name')
                if pd.isna(player_name_injury):
                     player_name_injury = row.get('display_name')
                if pd.isna(player_name_injury):
                     player_name_injury = row.get('gsis_id') # Fallback to gsis_id
                     if pd.isna(player_name_injury): continue

                week = row.get('week')
                report_status = row.get('report_status')


                if pd.notna(week) and week <= total_weeks: # Cap injury data to total_weeks
                     if player_name_injury not in player_injury_status:
                         player_injury_status[player_name_injury] = {}
                     # A player is 'Out' for this week if their 'report_status' is 'Out' or 'Doubtful'
                     is_out = (report_status == 'Out') or (report_status == 'Doubtful')
                     player_injury_status[player_name_injury][week] = is_out


        weekly_player_availability = {}
        # Use original names from aligned_players for consistency
        drafted_player_names = [p.get('original_name') for p in aligned_players if p.get('original_name')]


        for week_num in range(1, total_weeks + 1):
            weekly_player_availability[week_num] = {}
            for player_name in drafted_player_names:
                on_bye = player_bye_weeks.get(player_name) == week_num
                # Need to map player_name (original_name) to the name used in injury data (full_name or display_name) for accurate lookup
                # Iterate through aligned_players to find the full_name for the current player_name
                player_full_name_for_injury = player_name # Default to original_name
                for p in aligned_players:
                     if p.get('original_name') == player_name:
                         player_full_name_for_injury = p.get('full_name', player_name) # Use full_name if available
                         break


                is_injured_out = player_injury_status.get(player_full_name_for_injury, {}).get(week_num, False)

                is_playing = not on_bye and not is_injured_out

                weekly_player_availability[week_num][player_name] = is_playing


    except FileNotFoundError as e:
        print(f"Error loading data file: {e}. Please check the file paths.")
        return None
    except Exception as e:
        print(f"An error occurred during data loading and preparation: {e}")
        return None

    print("Data loaded and prepared.")

    # --- Team Drafting ---
    print("Running draft simulation...")
    # Pass aligned_players_sorted_by_adp to the draft simulation
    teams, picks = run_snake_draft(aligned_players_sorted_by_adp, num_teams, roster_limits, FLEX_ELIG, ROUNDS, RANDOM_PICK_RANGE)
    print_draft_board(picks, num_teams, ROUNDS, show_pos=True)

    # --- Initialize starting history for each player ---
    for team in teams:
        for player in team.roster:
            player['started_weeks'] = [] # Initialize an empty list to store starting history


    print("Draft simulation complete.")
    # print("Example drafted player:", teams[0].roster[0]) # Debugging


    # --- Schedule Generation ---
    print("Generating schedule...")
    def generate_schedule(teams, total_weeks):
        team_names = [f"Team {i+1}" for i in range(len(teams))]
        n = len(team_names)
        if n % 2 != 0:
            team_names.append("Bye_Team_Placeholder") # Use a distinct placeholder
            n += 1

        base_schedule = [[] for _ in range(n - 1)]
        mid = n // 2
        fixed_team = team_names[0]
        rotating_teams = team_names[1:]

        for round_num in range(n - 1):
            if rotating_teams[0] != "Bye_Team_Placeholder":
                base_schedule[round_num].append((fixed_team, rotating_teams[0]))
            for i in range(1, mid):
                team1 = rotating_teams[i]
                team2 = rotating_teams[n - 1 - i]
                if team1 != "Bye_Team_Placeholder" and team2 != "Bye_Team_Placeholder":
                    base_schedule[round_num].append((team1, team2))
            last_team = rotating_teams.pop()
            rotating_teams.insert(0, last_team)

        full_schedule = []
        for week in range(total_weeks):
            base_week_index = week % (n - 1)
            full_week_matches = list(base_schedule[base_week_index])
            # Remove matchups involving the Bye_Team_Placeholder if it exists
            full_week_matches = [match for match in full_week_matches if "Bye_Team_Placeholder" not in match]
            random.shuffle(full_week_matches)
            full_schedule.append(full_week_matches)

        return full_schedule

    generated_schedule = generate_schedule(teams, total_weeks)
    print("Schedule generated.")

    # --- Coach Bot Function ---
    print("Defining coach bot...")
    # set_optimal_lineup function needs to be defined or accessible here
    # Assuming set_optimal_lineup from cell 1b37f2a8 is available in the environment
    # It currently doesn't take roster_limits and flex_elig as arguments. Let's update it.
    def set_optimal_lineup(team_roster_aligned, weekly_player_availability, week_num, roster_limits, flex_elig):
        """
        Determines the optimal starting lineup for a fantasy football team based on
        player availability and projections for a given week.

        Args:
            team_roster_aligned (list): A list of player dictionaries for the team,
                                       including 'original_name', 'pos', and 'proj'.
            weekly_player_availability (dict): Nested dict: week -> player_name -> bool (available).
            week_num (int): The current week number for the simulation.
            roster_limits (dict): Dictionary defining roster limits by position.
            flex_elig (set): Set of positions eligible for FLEX.

        Returns:
            tuple: A tuple containing two lists: (starting_lineup, bench_players).
                   Each list contains player dictionaries.
        """
        available_players = []
        unavailable_players = []

        # Separate available and unavailable players for the current week
        for player in team_roster_aligned:
            player_name = player.get('original_name')
            if player_name: # Ensure player_name exists
                is_available = weekly_player_availability.get(week_num, {}).get(player_name, False)
                if is_available:
                    available_players.append(player)
                else:
                    unavailable_players.append(player)
            else:
                # Handle players without original_name if necessary (e.g., treat as unavailable)
                unavailable_players.append(player)


        # Sort available players by projection in descending order
        available_players.sort(key=lambda x: x.get('proj', 0), reverse=True)

        starting_lineup = []
        bench_players = []
        starters_count = {pos: 0 for pos in roster_limits.keys()} # Initialize counts for all positions


        # Fill required starting positions (QB, RB, WR, TE)
        required_positions = ["QB", "RB", "WR", "TE"]
        for pos in required_positions:
            needed = roster_limits.get(pos, 0)
            # Filter available players for the current required position
            pos_available = [p for p in available_players if p.get('pos') == pos]
            pos_available.sort(key=lambda x: x.get('proj', 0), reverse=True) # Sort by proj

            for player in pos_available:
                if starters_count[pos] < needed:
                    starting_lineup.append(player)
                    starters_count[pos] += 1
                    # Remove from available_players list (iterate over a copy or use indices if needed)
                    # To safely remove while iterating, we can build the remaining_available list
                else:
                    bench_players.append(player) # If not needed in starter, goes to bench


        # Build remaining available players list more safely
        remaining_available_after_required = []
        for player in available_players:
             if player not in starting_lineup:
                  remaining_available_after_required.append(player)


        # Fill FLEX position from remaining available players
        flex_eligible_players = [p for p in remaining_available_after_required if p.get('pos') in flex_elig]
        flex_needed = roster_limits.get("FLEX", 0)
        if starters_count.get("FLEX", 0) < flex_needed and flex_eligible_players:
            # FLEX is the highest projected among remaining eligible players
            flex_eligible_players.sort(key=lambda x: x.get('proj', 0), reverse=True)
            if flex_eligible_players:
                flex_player = flex_eligible_players[0]
                starting_lineup.append(flex_player)
                starters_count["FLEX"] += 1
                # Remove from remaining_available_after_required list
                if flex_player in remaining_available_after_required:
                    remaining_available_after_required.remove(flex_player)


        # All remaining available players go to the bench
        bench_players.extend(remaining_available_after_required)
        bench_players.extend(unavailable_players) # Add unavailable players to the bench as well


        return starting_lineup, bench_players


    print("Coach bot defined.")


    # --- Season Simulation ---
    print("Simulating season...")
    standings = {}
    weekly_matchup_data = [] # List to store weekly matchup details

    for i in range(num_teams):
        team_name = f"Team {i+1}"
        standings[team_name] = {'wins': 0, 'losses': 0, 'total_points': 0.0}


    for week_num, matchups in enumerate(generated_schedule, start=1):
        # print(f"\n--- Simulating Week {week_num} ---") # Optional: print weekly progress

        weekly_scores = {}
        weekly_matchups_this_week = [] # To store details for this week's matchups


        for match in matchups:
            team1_name, team2_name = match
            team1_index = int(team1_name.split(' ')[1])-1
            team2_index = int(team2_name.split(' ')[1])-1


            # Get the full drafted roster for each team (using the dictionaries stored in Team.roster)
            team1_full_roster = teams[team1_index].roster
            team2_full_roster = teams[team2_index].roster


            # Use the coach bot to determine the starting lineup for each team
            team1_starting_lineup, team1_bench = set_optimal_lineup(
                team1_full_roster, weekly_player_availability, week_num, roster_limits, FLEX_ELIG
            )
            team2_starting_lineup, team2_bench = set_optimal_lineup(
                team2_full_roster, weekly_player_availability, week_num, roster_limits, FLEX_ELIG
            )

            # --- Update started_weeks for Team 1 ---
            team1_starting_player_ids = {p.get('player_id') for p in team1_starting_lineup if p.get('player_id')}
            for player in team1_full_roster:
                 player_id = player.get('player_id')
                 if player_id in team1_starting_player_ids:
                     player['started_weeks'].append(1)
                 else:
                     player['started_weeks'].append(0)

            # --- Update started_weeks for Team 2 ---
            team2_starting_player_ids = {p.get('player_id') for p in team2_starting_lineup if p.get('player_id')}
            for player in team2_full_roster:
                 player_id = player.get('player_id')
                 if player_id in team2_starting_player_ids:
                     player['started_weeks'].append(1)
                 else:
                     player['started_weeks'].append(0)






















            # Calculate team scores based *only* on the players in the starting lineup
            team1_score = 0.0
            team1_starting_player_points = [] # Store points for starting players in Team 1
            for player in team1_starting_lineup:
                player_id = player.get('player_id')
                player_name = player.get('original_name', 'N/A')
                player_pos = player.get('pos', 'N/A')

                if player_id:
                    # Use the weekly_ppr_points_map for quick lookup by player_id and week
                    weekly_points = weekly_ppr_points_map.get(player_id, {}).get(week_num, 0.0)
                    team1_score += weekly_points
                    team1_starting_player_points.append({
                        'player_name': player_name,
                        'player_id': player_id,
                        'pos': player_pos,
                        'points': weekly_points
                    })
                else:
                    # Include players without an ID in the matchup data, but with 0 points
                     team1_starting_player_points.append({
                        'player_name': player_name,
                        'player_id': None,
                        'pos': player_pos,
                        'points': 0.0
                    })


            team2_score = 0.0
            team2_starting_player_points = [] # Store points for starting players in Team 2
            for player in team2_starting_lineup:
                player_id = player.get('player_id')
                player_name = player.get('original_name', 'N/A')
                player_pos = player.get('pos', 'N/A')

                if player_id:
                     # Use the weekly_ppr_points_map for quick lookup by player_id and week
                    weekly_points = weekly_ppr_points_map.get(player_id, {}).get(week_num, 0.0)
                    team2_score += weekly_points
                    team2_starting_player_points.append({
                        'player_name': player_name,
                        'player_id': player_id,
                        'pos': player_pos,
                        'points': weekly_points
                    })
                else:
                     # Include players without an ID in the matchup data, but with 0 points
                     team2_starting_player_points.append({
                        'player_name': player_name,
                        'player_id': None,
                        'pos': player_pos,
                        'points': 0.0
                    })


            weekly_scores[match] = (team1_score, team2_score)

            # Store matchup details
            weekly_matchups_this_week.append({
                'week': week_num,
                'team1': team1_name,
                'team2': team2_name,
                'team1_score': team1_score,
                'team2_score': team2_score,
                'team1_starters': team1_starting_player_points,
                'team2_starters': team2_starting_player_points
            })


        # Update standings based on weekly scores after all matchups for the week are processed
        for match, (score1, score2) in weekly_scores.items():
            team1_name, team2_name = match
            standings[team1_name]['total_points'] += score1

            # Only add total points for the second team if they are not the same team (handles potential bye placeholder issues)
            if team1_name != team2_name:
                 standings[team2_name]['total_points'] += score2


            if score1 > score2:
                standings[team1_name]['wins'] += 1
                # Only add loss for the second team if they are not the same team
                if team1_name != team2_name:
                    standings[team2_name]['losses'] += 1
            elif score2 > score1:
                standings[team2_name]['wins'] += 1
                # Only add loss for the first team if they are not the same team
                if team1_name != team2_name:
                    standings[team1_name]['losses'] += 1
            # Ties are not explicitly handled in wins/losses currently

        weekly_matchup_data.extend(weekly_matchups_this_week) # Add this week's matchups to the main list


    print("Season simulation complete.")

    # --- Generate and Return Final Standings ---
    print("Generating final standings...")
    # Sort by wins (descending), then losses (ascending), then total points (descending)
    sorted_standings_list = sorted(
        standings.items(),
        key=lambda item: (-item[1]['wins'], item[1]['losses'], -item[1]['total_points'])
    )

    # Convert back to dictionary for return value
    final_standings = dict(sorted_standings_list)

    print("--- Simulation Finished ---")
    # Return necessary data for counterfactual analysis and started_weeks, plus weekly_matchup_data
    return final_standings, teams, generated_schedule, weekly_ppr_points_map, weekly_player_availability, weekly_matchup_data

In [27]:
# Run the full simulation
# The run_full_simulation function now returns multiple artifacts in a tuple
artifacts = run_full_simulation(num_teams=12, total_weeks=17, weekly_stats_year=2023, injuries_year=2023)

# Check if the simulation ran successfully and returned artifacts
if artifacts is None:
    print("\nSimulation did not complete successfully.")
else:
    # Unpack the artifacts. The first element is the final standings dictionary.
    final_standings = artifacts[0]

    # Print the final standings if the simulation ran successfully
    if final_standings: # Check if the extracted standings dictionary is not empty
        print("\nFinal Standings:")
        # Iterate through the items of the standings dictionary
        for team, record in final_standings.items():
            print(f"{team}: Wins - {record['wins']}, Losses - {record['losses']}, Total Points - {record['total_points']:.2f}")
    else:
        print("\nNo final standings data available.")

--- Starting Fantasy Football Simulation ---
Loading and preparing data...
Downcasting floats.
Data loaded and prepared.
Running draft simulation...

=== Draft Board ===
                         Team 1                    Team 2               Team 3                Team 4                   Team 5                  Team 6                   Team 7                Team 8                    Team 9                   Team 10                   Team 11                  Team 12
Rnd 1     Justin Jefferson (WR)  Christian McCaffrey (RB)   Ja'Marr Chase (WR)       Nick Chubb (RB)        Travis Kelce (TE)        Tyreek Hill (WR)         Cooper Kupp (WR)    Austin Ekeler (RB)          CeeDee Lamb (WR)         Tony Pollard (RB)         Stefon Diggs (WR)   Amon-Ra St. Brown (WR)
Rnd 2        Jaylen Waddle (WR)          Jalen Hurts (QB)      Josh Allen (QB)  Jonathan Taylor (RB)         Josh Jacobs (RB)      Derrick Henry (RB)  Patrick Mahomes II (QB)   Bijan Robinson (RB)       Saquon Barkley (RB)       G

In [28]:
# Ensure simulation artifacts are available
if 'artifacts' not in locals() or artifacts is None:
    print("Simulation artifacts not found. Please run the simulation first.")
else:
    # Extract the weekly_matchup_data from the artifacts tuple
    weekly_matchup_data = artifacts[5]

    # Display the weekly_matchup_data as a pandas DataFrame for better visibility
    if weekly_matchup_data:
        weekly_matchup_df = pd.DataFrame(weekly_matchup_data)
        print("\nWeekly Matchup Data:")
        display(weekly_matchup_df)
    else:
        print("\nNo weekly matchup data available.")


Weekly Matchup Data:


,week,team1,team2,team1_score,team2_score,team1_starters,team2_starters
0,1,Team 6,Team 9,76.600000,80.840000,"[{'player_name': 'Lamar Jackson', 'player_id':...","[{'player_name': 'Justin Fields', 'player_id':..."
1,1,Team 5,Team 10,38.860001,93.860000,"[{'player_name': 'Justin Herbert', 'player_id'...","[{'player_name': 'Deshaun Watson', 'player_id'..."
2,1,Team 1,Team 2,69.980000,73.199999,"[{'player_name': 'Joe Burrow', 'player_id': '0...","[{'player_name': 'Jalen Hurts', 'player_id': '..."
3,1,Team 7,Team 8,63.000001,99.939999,"[{'player_name': 'Patrick Mahomes II', 'player...","[{'player_name': 'Trevor Lawrence', 'player_id..."
4,1,Team 4,Team 11,48.680001,94.740000,"[{'player_name': 'Geno Smith', 'player_id': '0...","[{'player_name': 'Tua Tagovailoa', 'player_id'..."
...,...,...,...,...,...,...,...
97,17,Team 2,Team 3,97.380001,73.460000,"[{'player_name': 'Jalen Hurts', 'player_id': '...","[{'player_name': 'Josh Allen', 'player_id': '0..."
98,17,Team 11,Team 5,107.579999,20.100000,"[{'player_name': 'Tua Tagovailoa', 'player_id'...","[{'player_name': 'Justin Herbert', 'player_id'..."
99,17,Team 9,Team 7,101.520000,47.000000,"[{'player_name': 'Justin Fields', 'player_id':...","[{'player_name': 'Patrick Mahomes II', 'player..."
100,17,Team 1,Team 8,41.399999,50.200001,"[{'player_name': 'Joe Burrow', 'player_id': '0...","[{'player_name': 'Austin Ekeler', 'player_id':..."


In [29]:
import numpy as np

def normalized_method(player, weekly_points, x):
    """
    Adds an amount 'x' to a player's weekly points for the weeks they started,
    proportional to the points scored in those started weeks relative to the
    total points scored in all started weeks.

    Args:
        player (dict): The player dictionary, including 'started_weeks'.
        weekly_points (dict): A dictionary of week -> points for the player.
        x (float): The total amount to distribute over the started weeks.

    Returns:
        dict: A dictionary of week -> points with the proportional amount added.
    """
    started_weeks_list = player.get('started_weeks', [])
    total_started_weeks_count = sum(started_weeks_list)

    if total_started_weeks_count == 0:
        print(f"Warning: {player.get('original_name', 'Player')} did not start any weeks. No points added.")
        return weekly_points.copy() # Return a copy of original points if no weeks started

    # Get the points scored in the weeks the player actually started
    points_in_started_weeks = {}
    for week, started in enumerate(started_weeks_list, start=1):
        if started == 1:
            # Ensure the week exists in the weekly_points dictionary before getting points
            points_in_started_weeks[week] = weekly_points.get(week, 0.0)


    # Calculate the total points scored *only* in the started weeks
    total_points_in_started_weeks = sum(points_in_started_weeks.values())

    # Create a new dictionary for modified points, starting with original points
    modified_weekly_points = weekly_points.copy()

    # Ensure total points in started weeks is not zero to avoid division by zero
    if total_points_in_started_weeks <= 0: # Use <= 0 to also handle negative sums if that were ever possible
         print(f"Warning: {player.get('original_name', 'Player')} had 0 or negative total points in started weeks. Cannot apply proportional method.")
         return weekly_points.copy()


    # Distribute x proportionally among the started weeks based on points scored
    for week, points_scored_in_week in points_in_started_weeks.items():
        # Calculate proportional amount based on points in that week relative to total points in started weeks
        proportion_of_total = points_scored_in_week / total_points_in_started_weeks
        proportional_amount = proportion_of_total * x
        modified_weekly_points[week] += proportional_amount

    return modified_weekly_points

In [30]:
def basic_method(player, weekly_points, x):
    """
    Adds an amount 'x' to a player's weekly points for the weeks they started,
    by dividing x evenly among those started weeks.

    Args:
        player (dict): The player dictionary, including 'started_weeks'.
        weekly_points (dict): A dictionary of week -> points for the player.
        x (float): The total amount to distribute over the started weeks.

    Returns:
        dict: A dictionary of week -> points with the averaged amount added.
    """
    started_weeks_list = player.get('started_weeks', [])
    total_started_weeks_count = sum(started_weeks_list)

    if total_started_weeks_count == 0:
        print(f"Warning: {player.get('original_name', 'Player')} did not start any weeks. No points added.")
        return weekly_points.copy() # Return a copy of original points if no weeks started

    # Calculate the amount to add per started week (even distribution)
    amount_per_started_week = x / total_started_weeks_count

    # Create a new dictionary for modified points, starting with original points
    modified_weekly_points = weekly_points.copy()

    # Add the averaged amount to the points for each started week
    for week, started in enumerate(started_weeks_list, start=1):
        if started == 1:
            # Ensure the week exists in the modified_weekly_points dictionary before adding
            if week in modified_weekly_points:
                modified_weekly_points[week] += amount_per_started_week
            else:
                # If a started week is not in modified_weekly_points (e.g., bye week, injury with 0 points),
                # initialize it with the averaged amount.
                modified_weekly_points[week] = amount_per_started_week

    return modified_weekly_points

In [68]:
import random

def finite_element_alalysis(artifacts, delta_x):
    # Ensure simulation artifacts are available
    if 'artifacts' not in locals() or artifacts is None:
        print("Simulation artifacts not found. Please run the simulation first.")
        return  # Exit the function if artifacts are not found

    teams = artifacts[1]
    weekly_ppr_points_map = artifacts[3]  # This is the map of (player_id, week) -> points
    original_standings = artifacts[0]     # Get the original standings
    weekly_matchup_data = artifacts[5]    # Get the weekly matchup data

    # Get the roster for Team 1 (index 0)
    if len(teams) > 0:
        team1_roster = teams[0].roster

        # Filter for players who were started in at least one week and have a player_id
        starting_players = [
            player for player in team1_roster
            if player.get('started_weeks') and any(player['started_weeks']) and player.get('player_id')
        ]

        # Select a random player from the starting players
        if starting_players:
            random_starter = random.choice(starting_players)
            player_id_to_find = random_starter.get('player_id')
            player_name_to_print = random_starter.get('original_name', 'N/A')

            # Check if the selected player has weekly points data
            if (player_id_to_find not in weekly_ppr_points_map or
                    not weekly_ppr_points_map[player_id_to_find]):
                print(
                    f"\nWarning: Weekly point data not found for randomly selected "
                    f"player {player_name_to_print} (Player ID: {player_id_to_find}). "
                    "Skipping analysis for this player."
                )
                return  # Exit the function if no weekly points data

            print(
                f"\nSelected Random Starting Player from Team 1: "
                f"{player_name_to_print} ({random_starter.get('pos', 'N/A')})"
            )

        else:
            print("\nNo players from Team 1 were started in any week with available "
                  "player IDs and started weeks data.")
            return  # Exit the function if no starting players found
    else:
        print("\nNo teams found in simulation artifacts.")
        return  # Exit the function if no teams found

    # If we reach here, random_starter and its weekly points should be available
    player_to_modify = random_starter
    player_id_to_find = player_to_modify.get('player_id')
    player_name_to_print = player_to_modify.get('original_name', 'N/A')

    # Get the original points
    player_points = weekly_ppr_points_map[player_id_to_find]
    x_value = delta_x  # Use the delta_x passed to the function

    # Apply the normalized_method to get modified points
    # This function should distribute x_value total points across started weeks
    modified_points = normalized_method(player_to_modify, player_points, x_value)

    print(f"\nOriginal weekly points for {player_name_to_print}:")
    original_sorted_weeks = sorted(player_points.keys())
    if original_sorted_weeks:
        for week in original_sorted_weeks:
            points = player_points.get(week, 0.0)
            print(f"Week {week}: {points:.2f}")
    else:
        print("Original weekly point data not found for this player.")

    print(
        f"\nModified weekly points for {player_name_to_print} "
        f"(with {x_value} points normalized over started weeks):"
    )
    modified_sorted_weeks = sorted(modified_points.keys())
    if modified_sorted_weeks:
        for week in modified_sorted_weeks:
            points = modified_points.get(week, 0.0)
            print(f"Week {week}: {points:.2f}")
    else:
        print("Modified weekly point data not found for this player.")

    # Find the team index of the random_starter
    team_index_with_player = -1
    for i, team in enumerate(teams):
        if any(p.get('player_id') == player_id_to_find for p in team.roster):
            team_index_with_player = i
            break

    if team_index_with_player == -1:
        print(f"\nCould not find the team for player {player_name_to_print}.")
        return  # Exit if team not found
    else:
        team_name_to_analyze = f"Team {team_index_with_player + 1}"
        print(f"\nAnalyzing the impact on {team_name_to_analyze}'s record.")

        # Create a copy of the weekly matchup data to avoid modifying the original
        modified_matchup_data = weekly_matchup_data.copy()

        # Initialize modified wins and losses for the team
        modified_wins = 0
        modified_losses = 0
        modified_total_points = 0.0  # Initialize total points

        # Iterate through the copied matchup data to apply modified points
        #  weekly_ppr_points_map = artifacts[3]  # This is the map of (player_id, week) -> points
        for matchup in modified_matchup_data:
            week_num = matchup['week']
            team1_name = matchup['team1']
            team2_name = matchup['team2']

            # Check if the team being analyzed is involved in this matchup
            if team1_name == team_name_to_analyze or team2_name == team_name_to_analyze:
                team1_starters = matchup['team1_starters']
                team2_starters = matchup['team2_starters']

                score_recalculated_this_matchup = False

                # Check if the player is in Team 1's starting lineup
                if team1_name == team_name_to_analyze:
                    for player_data in team1_starters:
                        if player_data.get('player_id') == player_id_to_find:
                            player_data['points'] = modified_points.get(
                                week_num,
                                player_data.get('points', 0.0)
                            )

                            score_recalculated_this_matchup = True
                            break

                # Check if the player is in Team 2's starting lineup
                if team2_name == team_name_to_analyze:
                    for player_data in team2_starters:
                        if player_data.get('player_id') == player_id_to_find:
                            player_data['points'] = modified_points.get(
                                week_num,
                                player_data.get('points', 0.0)
                            )
                            score_recalculated_this_matchup = True
                            break

                # Recalculate the team's score if the player's points were modified
                if score_recalculated_this_matchup:
                    if team1_name == team_name_to_analyze:

                        matchup['team1_score'] = sum(p['points'] for p in team1_starters)
                        print(matchup['team1_score'])

                    if team2_name == team_name_to_analyze:
                        print(matchup['team2_score'])
                        matchup['team2_score'] = sum(p['points'] for p in team2_starters)

        # After applying modified points to relevant matchups,
        # determine the new wins, losses, and total points
        for matchup in modified_matchup_data:
            team1_name = matchup['team1']
            team2_name = matchup['team2']
            team1_score = matchup['team1_score']
            team2_score = matchup['team2_score']

            if team1_name == team_name_to_analyze:
                modified_total_points += team1_score
                if team1_score > team2_score:
                    modified_wins += 1
                elif team2_score > team1_score:
                    modified_losses += 1
            elif team2_name == team_name_to_analyze:
                modified_total_points += team2_score
                if team2_score > team1_score:
                    modified_wins += 1
                elif team1_score > team2_score:
                    modified_losses += 1

        # Get the original record
        original_record = original_standings.get(
            team_name_to_analyze,
            {'wins': 0, 'losses': 0, 'total_points': 0.0}
        )

        print(
            f"\nOriginal Record for {team_name_to_analyze}: "
            f"{original_record['wins']} Wins, {original_record['losses']} Losses, "
            f"{original_record['total_points']:.2f} Total Points"
        )
        print(
            f"Modified Record for {team_name_to_analyze}: "
            f"{modified_wins} Wins, {modified_losses} Losses, "
            f"{modified_total_points:.2f} Total Points"
        )

        return modified_wins - original_record['wins']


In [69]:
delta_wins = finite_element_alalysis(artifacts, 50)
print(delta_wins)


Selected Random Starting Player from Team 1: Tyler Lockett (WR)

Original weekly points for Tyler Lockett:
Week 1: 3.00
Week 2: 25.90
Week 3: 8.40
Week 4: 9.40
Week 6: 15.40
Week 7: 7.80
Week 8: 22.10
Week 9: 6.20
Week 10: 23.20
Week 11: 10.10
Week 12: 6.00
Week 13: 9.70
Week 14: 14.90
Week 15: 5.10
Week 16: 16.10
Week 17: 2.00

Modified weekly points for Tyler Lockett (with 50 points normalized over started weeks):
Week 1: 3.81
Week 2: 32.89
Week 3: 10.67
Week 4: 11.94
Week 6: 19.56
Week 7: 9.90
Week 8: 28.06
Week 9: 7.87
Week 10: 29.46
Week 11: 12.83
Week 12: 7.62
Week 13: 12.32
Week 14: 18.92
Week 15: 6.48
Week 16: 20.44
Week 17: 2.54

Analyzing the impact on Team 1's record.
94.84820074101371
125.66813046119705
123.64449334542066
113.9400218504183
87.23888413271958
49.162307128458345
123.70869979058995
72.28121530761189
86.50405186993142
50.07773258683943
87.43614759276629
84.02469508182068
54.29716430812681
73.33741103148165
84.48967864576387
67.96475531978264

Original Record fo